# Required Scripts

In [264]:
import os
import pandas as pd
import numpy as np

from pandas import ExcelWriter
from pandas import ExcelFile

# Calculate Results

In [265]:
dirs = os.listdir('.') 

# This will print all the files and directories
for file in dirs:
    print file

# response = None
while response not in dirs:
    response = raw_input("Please enter filename: ")

response = 'Bearclave.xlsx'
df = pd.read_excel(response, sheetname=None)

.ipynb_checkpoints
Bearclave Schedule_Scoresheet 2017.xlsx
Bearclave.xlsx
BearClaveScheduleONLY.xlsx
CAL Registration Bearclave 2017.xlsx
ConclaveCalc.html
ConclaveCalc.ipynb
ConclaveCalc.py
CPL Registration Bearclave 2017.xlsx
HSU Registration Bearclave 2017.xlsx
Untitled.html
Untitled.py
Untitled.txt
~$Bearclave Schedule_Scoresheet 2017.xlsx
~$Bearclave.xlsx


In [321]:
# Assigns rank based on ascending or descending scores
pt_order ={
    'Dendro': False,
    'Traverse': True,
    'Hard': False,
    'Double': True,
    'PSaw': True,
    'Choker': True,
    'Throw': False,
    'Single': True,
    'Caber': True,
    'Climb': False,
    'JackJill': True,
    'Speed': True,
    'Pulp': True,
    'OP': True}

# Assigns points to rankings
rankswap = {1:10, 2:7, 3:5, 4:3, 5:1}

# Assigns points based on whether events are run separately for men and women
gender_split = ['Dendro', 'Traverse', 'Cruise']

# Assigns whether or not events have a partner
partner_split = ['Double', 'JackJill', 'Caber']

In [325]:
scores = pd.DataFrame()

for key, value in df.iteritems() :
    
    if key in gender_split:
        temp = df[key].dropna(subset=['Contestant'])
        tempN = temp.sort_values('Score',ascending = pt_order[key]).reset_index(drop=True)
        # Assign rank to competitor names
        tempN['Rank'] = tempN.index + 1
        # Assign point values to rank
        tempN['Points'] = tempN.Rank.map(rankswap)
        tempN = tempN[tempN['Points'] >0]
        tempN['Event'] = key
        scores = tempN.append(scores)
#         print key
    
    else:
        # Clean Input Data
        temp = df[key].dropna(subset=['Contestant'])
    
        # Sort based on correlated rank
        tempM = temp[temp['Gender']=='M'].sort_values('Score',ascending = pt_order[key]).reset_index(drop=True)
        # Assign rank to competitor names
        tempM['Rank'] = tempM.index + 1
        # Assign point values to rank
        tempM['Points'] = tempM.Rank.map(rankswap)
        tempM = tempM[tempM['Points'] >0]
        tempM['Event'] = key
        scores = tempM.append(scores)
#         print key + '-M'
        
        # Sort based on correlated rank
        tempF = temp[temp['Gender']=='F'].sort_values('Score',ascending = pt_order[key]).reset_index(drop=True)
        # Assign rank to competitor names
        tempF['Rank'] = tempF.index + 1
        # Assign point values to rank
        tempF['Points'] = tempF.Rank.map(rankswap)
        tempF = tempF[tempF['Points'] >0]
        tempF['Event'] = key
        scores = tempF.append(scores)
#         print key + '-F'

In [326]:
# Clean output
# Weigh scores in partner events
scores['multiplier'] = np.where(scores['Contestant B'].notnull(), 0.5, 1)
scores['Points'] = scores['multiplier']*scores['Points']
scores = scores[['Contestant', 'Gender', 'Contestant B', 'Gender B', 'Event', 'Team', 'School', 'Points', 'Rank']]

# isolate partners
partners = scores[scores['Contestant B'].notnull()]
partners.drop(['Contestant', 'Gender'], axis=1, inplace=True)
partners.rename(columns={'Contestant B': 'Contestant', 'Gender B': 'Gender'}, inplace=True)

# Clean remaining results from contestants
contestants = scores
contestants.drop(['Contestant B', 'Gender B'], axis=1, inplace=True)

# Recombine competitors and partners into final scoresheet
scores = contestants.append(partners)

c:\users\risato\anaconda2\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Print Results

In [318]:
# Print Top 3 in each event
print scores[scores['Rank']<=3].sort_values(['Event', 'Gender', 'Rank'])

           Contestant Gender     Event Team School  Points  Rank
0               becca      F     Caber    B    HSU    10.0     1
1        Jillian zuck      F     Caber    B    CPL     7.0     2
2      Rachel Werhman      F     Caber    A    CPL     5.0     3
0     Charlie Watkins      M     Caber    B    CPL    10.0     1
1             brian D      M     Caber    A    HSU     7.0     2
2              b wall      M     Caber    B    HSU     5.0     3
0              Dana G      F    Choker    A    HSU    10.0     1
1                Naya      F    Choker    B    HSU     7.0     2
2    Giselle Vandrick      F    Choker    B    UCB     5.0     3
0       Rory MacNeill      M    Choker    A    CPL    10.0     1
1             Brian D      M    Choker    A    HSU     7.0     2
2                Sean      M    Choker    A    HSU     5.0     3
1     Kathie Pauloski      F    Dendro    A    UCB     7.0     2
2   Kateyln Sunderman      F    Dendro    A    CPL     5.0     3
0        Jacob Levine    

In [292]:
# Print "Bull" and "Bell" of the woods
report = scores.groupby(['Contestant', 'School', 'Team','Gender'],  as_index=False).sum().sort_values('Points', ascending = False)

print 'Bull'
print report[report['Gender'] == 'M'].reset_index(drop = True).loc[0]
print '\n'
print 'Bell'
print report[report['Gender'] == 'F'].reset_index(drop = True).loc[0]

Bull
Contestant    Alex
School         HSU
Team             B
Gender           M
Points          23
Rank            10
Name: 0, dtype: object


Bell
Contestant    Dana G
School           HSU
Team               A
Gender             F
Points            20
Rank               2
Name: 0, dtype: object


In [282]:
# Print Team Rankings
report = final.groupby(['School', 'Team'],  as_index=False).sum().sort_values('Points', ascending = False)
print report  

  School Team  Points  Rank
2    HSU    A   166.0    76
0    CPL    A   125.0    70
3    HSU    B    92.0    75
4    UCB    A    85.0    71
1    CPL    B    82.0    85
5    UCB    B    17.0    24
